In [ ]:
%run yulesimon-lib.ipynb

# CONSTANTS

In [ ]:
# N_RANGE=[6,7,8,9]; RANDOM_STATES=[0, 17]
N_RANGE=range(5, 12); RANDOM_STATES=[0, 3, 5, 8, 11, 16, 17, 20, 21, 24]

trials_out = []

RUN_DNN = True
RUN_DNN_ZEROS = True
RUN_DNN_WIN = True
RUN_DNN_WIN_ZEROS = True
RUN_CNN = True
RUN_CNN_ZEROS = True
RUN_CNNM = False


# DNN

In [ ]:
if RUN_DNN:
    trials_out.append(trial(nn='DNN', N_range=N_RANGE, random_states=RANDOM_STATES))

# DNN zeros

In [ ]:
if RUN_DNN_ZEROS:
    trials_out.append(trial(nn='DNN_ZEROS', N_range=N_RANGE, hstack_zeros=True, random_states=RANDOM_STATES))

# DNN win

In [ ]:
if RUN_DNN_WIN:
    trials_out.append(trial(nn='DNN_WIN', N_range=N_RANGE, random_states=RANDOM_STATES))

# DNN win zeros

In [ ]:
if RUN_DNN_WIN_ZEROS:
    trials_out.append(trial(nn='DNN_WIN_ZEROS', N_range=N_RANGE, hstack_zeros=True, random_states=RANDOM_STATES))

# CNN

In [ ]:
if RUN_CNN:
    trials_out.append(trial(nn='CNN', N_range=N_RANGE, random_states=RANDOM_STATES))

# CNN zeros

In [ ]:
if RUN_CNN_ZEROS:
    trials_out.append(trial(nn='CNN_ZEROS', N_range=N_RANGE, hstack_zeros=True, random_states=RANDOM_STATES))

# CNN multi-layer

In [ ]:
# TODO
if RUN_CNNM:
    trials_out.append(trial(nn='CNNM', N_range=N_RANGE, random_states=RANDOM_STATES))

# plot log(N) vs sqrt_mse

In [ ]:
def plot_sqrt_mse():
    
    plt.figure(figsize=(15, 4))
    
    ax1 = plt.subplot(1, 2, 1)

    ax1.set(title='sqrt_mse', xlabel='log(N)', ylabel='sqrt_mse')

    colors = ['blue', 'red', 'green', 'orange', 'purple']
    for i, out in enumerate(trials_out):
        x = np.log10(out['a_N'])
        y = out['a_sqrt_mse']
        std = out['a_std_abs_errors']
        ax1.plot(x, y, linewidth=2, color=colors[i], marker='o', markersize=6, label=out['nn'])
        ax1.fill_between(x, y - std, y + std, facecolor=colors[i], alpha=0.3)
    
    plt.legend(loc='upper right')

plot_sqrt_mse()


# print and save statistics

In [ ]:
dict_list = []
for t in trials_out:
    dict_list.append(('N', t['a_N']))
for t in trials_out:
    dict_list.append(('sqrt_mse_{}'.format(t['nn']), t['a_sqrt_mse']))
for t in trials_out:
    dict_list.append(('std_{}'.format(t['nn']), t['a_std_abs_errors']))
for t in trials_out:
    dict_list.append(('avg_abs_error_{}'.format(t['nn']), t['a_avg_abs_errors']))

df = pd.DataFrame(dict(dict_list))

df = df.set_index('N')

df.to_csv('yulesimon.csv')

columns=[]
for t in trials_out:
    columns.append(('sqrt_MSE',t['nn']))
for t in trials_out:
    columns.append(('STD',t['nn']))
for t in trials_out:
    columns.append(('avg_abs_err',t['nn']))

df.columns=pd.MultiIndex.from_tuples(columns)

print('err = y_test - y_pred')
df